In [0]:
#Installs dependencies
!pip install --upgrade -q pygsheets

import pygsheets

In [0]:
def createTestSheet(numBooks = 10):
    '''
    Creates a test sheet
    '''
    AUTHFILE = 'libraryKeyDev.json'
    MAINGOOGLEACCOUNT = ''
    gc = pygsheets.authorize(service_file=AUTHFILE)
    testColTitles = ['Title', 'Author', 'SUID', 'SUNet', 'Cell Phone', 'Address', 'Date Out', 'Date Due', 'Is Checked Out']
    numTitles = len(testColTitles)
    testBooks = []
    for idx in range(1, numBooks + 1):
        bookInfo = []
        for val in testColTitles[:len(testColTitles) - 1]:
            bookInfo.append(val + " " + str(idx))
        bookInfo.append("False")
        testBooks.append(bookInfo)

    sh = gc.sheet.create('testSheet')
    sh = gc.open('testSheet')
    wks = sh.sheet1
    #Writes Column Names
    colTitles = pygsheets.DataRange(start = (1, 1), 
                                    end = (1, numTitles),
                                    worksheet = wks)
    colTitles.update_values([testColTitles])
    #Writes Book Information
    books = pygsheets.DataRange(start = (2, 1), 
                                    end = (numBooks + 1, numTitles),
                                    worksheet = wks)
    books.update_values(testBooks)
    #Shares sheet to main google account
    sh.share(MAINGOOGLEACCOUNT, role='writer', type='user')


createTestSheet()

In [0]:
from collections import OrderedDict 
from datetime import date, timedelta
import pygsheets

AUTHFILE = 'libraryKeyDev.json'
LIBRARYINFO = "testSheet"
LIBRARYPAGE = 0
PERMHEADERS = ['Title', 'Author']
CHECKOUTLIMIT = 2

#TODO: Handle situation where book does not exist
#TODO: Lookup by title and author
#worksheet is named LIB
#colheaders are named LIBHEADERS

def getLibraryInfo():
    gc = pygsheets.authorize(service_file=AUTHFILE)
    sh = gc.open(LIBRARYINFO)
    lib = sh[LIBRARYPAGE]
    libHeaders = OrderedDict()
    fields = lib.get_row(1, include_tailing_empty = False)
    for idx, field in enumerate(fields):
        #Worksheet is 1 indexed
        libHeaders[field] = idx + 1
    return (lib, libHeaders)

def getPropertyColNum(prop):
    global LIBHEADERS
    try:
        return LIBHEADERS[prop]
    except KeyError: #TODO: Handle exception in program
        return -1

def getBookInfo(bookTitle, returnAs = 'matrix'):
    '''
    returnAs #=> 'matrix', 'cell', 'range'
    '''
    global LIB
    colNum = getPropertyColNum('Title')
    query = LIB.find(bookTitle, matchEntireCell = True, cols = (colNum, colNum))
    if len(query) == 0: #TODO: Handle situation where book does not exist
        print("No book with that title found.")
    elif len(query) > 1:
        print("Multiple books with that title found.")
    bookRow = query[0].row
    book = LIB.get_row(bookRow, returnas = returnAs, include_tailing_empty = False)
    return book

def getBookProperty(bookLookupInfo, prop):
    global LIB
    if isinstance(bookLookupInfo, str):
        titleCol = getPropertyColNum('Title')
        query = LIB.find(bookLookupInfo, matchEntireCell = True, cols = (titleCol, titleCol))
        if len(query) == 0: #TODO: Handle situation where book does not exist
            print("No book with that title found.")
        elif len(query) > 1:
            print("Multiple books with that title found.")
        rowNum = query[0].row
    else:
        rowNum = bookLookupInfo
    colNum = getPropertyColNum(prop)
    propValue = LIB.get_value((rowNum, colNum))
    return propValue

def checkOutBook(bookTitle, *addInfo):
    '''
    *addInfo #=> suID, suNet, cellPhone, address
    '''
    global PERMHEADERS
    #TODO: *addInfo should be well formed (includes all values)
    try:
        validateAddInfo()
    except INVALID
    book = getBookInfo(bookTitle, 'range')
    bookRow = book.start_addr[0]
    permBookInfo = []
    for prop in PERMHEADERS:
        propValue = getBookProperty(bookRow, prop)
        permBookInfo.append(propValue)
    checkoutInfo = getCheckoutInfo(permBookInfo, *addInfo)
    book.update_values([checkoutInfo])
    print(bookTitle + " has been checked out.")

def getCheckOutRange():
    today = date.today()
    dateOut = today.strftime("%d %B %Y")
    dateDue = today + timedelta(weeks = CHECKOUTLIMIT)
    dateDue = dateDue.strftime("%d %B %Y")
    return (dateOut, dateDue)

def getCheckoutInfo(permBookInfo, *addInfo):
    checkoutInfo = []
    for info in permBookInfo:
        checkoutInfo.append(info)
    for info in addInfo:
        checkoutInfo.append(info)
    dateOut, dateDue = getCheckOutRange()
    checkoutInfo.append(dateOut)
    checkoutInfo.append(dateDue)
    checkoutInfo.append('True')
    return checkoutInfo

def renewCheckOut(bookTitle):
    global LIB
    book = getBookInfo(bookTitle, 'range')
    bookRow = book.start_addr[0]
    dueDateCol = getPropertyColNum('Date Due')
    dateDue = getCheckOutRange()[1]
    LIB.update_value((bookRow, dueDateCol), str(dateDue))
    print(bookTitle + " is now due on " + dateDue + ".")

def checkInBook(bookTitle):
    global LIB
    global PERMHEADERS
    book = getBookInfo(bookTitle, 'range')
    bookRow = book.start_addr[0]
    permBookInfo = []
    for prop in PERMHEADERS:
        propValue = getBookProperty(bookRow, prop)
        permBookInfo.append(propValue)
    checkinInfo = getCheckInInfo(permBookInfo)
    book.update_values([checkinInfo])
    print(bookTitle + " has been checked in.")

def getCheckInInfo(permBookInfo):
    global LIBHEADERS
    global PERMHEADERS
    #-1 Corresponds to isCheckedOut Field
    extraneousFieldLen = len(LIBHEADERS.keys()) - len(PERMHEADERS) - 1
    checkinInfo = permBookInfo + [''] * extraneousFieldLen
    checkinInfo.append('False')
    return checkinInfo

def main():
    createTestSheet()
    global LIB
    global LIBHEADERS
    LIB, LIBHEADERS = getLibraryInfo()
    print('getPropertyColNum Tests:')
    print(getPropertyColNum('Title'))
    print(getPropertyColNum('rats'))
    print()
    print('getBookInfo Tests:')
    print(getBookInfo('Title 1'))
    print(getBookInfo('Title 2'))
    print(getBookInfo('Title 3'))
    print()
    print('getBookProperty Tests:')
    print(getBookProperty('Title 1', 'Title'))
    print(getBookProperty('Title 2', 'Title'))
    print(getBookProperty('Title 3', 'Title'))
    print(getBookProperty(2, 'Title'))
    print(getBookProperty(3, 'Title'))
    print(getBookProperty(4, 'Title'))
    print()
    print('checkOutBook Tests:')
    checkOutBook('Title 1', 123, 'testSUNET', 2214324536, 'Somewhere')
    print()
    print('renewCheckOut Tests:')
    renewCheckOut('Title 3')
    print()
    print('checkInBook Tests: ')
    checkInBook('Title 2')
    print("Run done.")

main()

getPropertyColNum Tests:
1
-1

getBookInfo Tests:
[<Cell A1 'Title'>, <Cell A2 'Title 1'>, <Cell A3 'Title 2'>, <Cell A4 'Title 3'>, <Cell A5 'Title 4'>, <Cell A6 'Title 5'>, <Cell A7 'Title 6'>, <Cell A8 'Title 7'>, <Cell A9 'Title 8'>, <Cell A10 'Title 9'>, <Cell A11 'Title 10'>, <Cell A12 ''>, <Cell A13 ''>, <Cell A14 ''>, <Cell A15 ''>, <Cell A16 ''>, <Cell A17 ''>, <Cell A18 ''>, <Cell A19 ''>, <Cell A20 ''>, <Cell A21 ''>, <Cell A22 ''>, <Cell A23 ''>, <Cell A24 ''>, <Cell A25 ''>, <Cell A26 ''>, <Cell A27 ''>, <Cell A28 ''>, <Cell A29 ''>, <Cell A30 ''>, <Cell A31 ''>, <Cell A32 ''>, <Cell A33 ''>, <Cell A34 ''>, <Cell A35 ''>, <Cell A36 ''>, <Cell A37 ''>, <Cell A38 ''>, <Cell A39 ''>, <Cell A40 ''>, <Cell A41 ''>, <Cell A42 ''>, <Cell A43 ''>, <Cell A44 ''>, <Cell A45 ''>, <Cell A46 ''>, <Cell A47 ''>, <Cell A48 ''>, <Cell A49 ''>, <Cell A50 ''>, <Cell A51 ''>, <Cell A52 ''>, <Cell A53 ''>, <Cell A54 ''>, <Cell A55 ''>, <Cell A56 ''>, <Cell A57 ''>, <Cell A58 ''>, <Cell A59 ''

In [0]:
class BookPropertyError(Exception):
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

In [0]:
def deleteAllFiles():
    AUTHFILE = 'libraryKeyDev.json'
    gc = pygsheets.authorize(service_file=AUTHFILE)
    for file in gc.open_all():
        print("File Deleted: " + file.title)
        file.delete()
    print("All files deleted.")


deleteAllFiles()

File Deleted: testSheet
All files deleted.


In [0]:
def printAllFileNames():
    AUTHFILE = 'libraryKeyDev.json'
    gc = pygsheets.authorize(service_file=AUTHFILE)
    for file in gc.open_all():
        print("File found: " + file.title)
    print("All file names printed.")


printAllFileNames()

File found: testSheet
All file names printed.
